## CrypTen - Private Model Inference using Plans


We will do a simple inference on an encrypted neural network that is not known by the local worker.
The workers that known the model structure are deployed as [GridNodes](https://github.com/OpenMined/PyGridNode). For this we will be using Plans and we will be using CrypTen as a backend for SMPC. 


Authors:
 - George Muraru - Twitter: [@gmuraru](https://twitter.com/georgemuraru)
 - Ayoub Benaissa - Twitter: [@y0uben11](https://twitter.com/y0uben11)

## Setup

### Download/install needed repos
* Install PySyft from the [`crypten` branch](https://github.com/OpenMined/PySyft/tree/crypten).
* Clone the [GridNode repository](https://github.com/OpenMined/GridNode)
  * we need this because *alice* and *bob* are two different GridNodes

### Run the grid nodes
* In two separate terminals run:
    * ```python -m gridnode --id alice --port 3000```
    * ```python -m gridnode --id bob --port 30001```
    
This will start two workers, *alice* and *bob* and we will connect to them using the port 3000 and 30001.

## Prepare the ground

In [ ]:
import pytest
import crypten

import torch
import torch.nn as nn
import torch.nn.functional as F
from time import time

import syft as sy

from syft.workers.node_client import NodeClient
from syft.frameworks.crypten.context import run_multiworkers

hook = sy.TorchHook(torch)

## Neural network that will be known only to the workers

In [ ]:
class ExampleNet(nn.Module):
    def __init__(self):
        super(ExampleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=0)
        self.fc1 = nn.Linear(16 * 12 * 12, 100)
        self.fc2 = nn.Linear(100, 2)
 
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 16 * 12 * 12)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out

## Setup the workers and send them the neural network

We need to have the two GridNodes workers running.

In our current scenario we are sending the serialized model to the workers that are taking part in the computation, but in a real life situation this sending part should not exist - we only need to know that we have the same model on all the workers.

### Scenario
* The local worker wants to run inference on the data that is hosted on *alice* machine.
* The model is known only by *alice* and *bob*

In [ ]:
# Syft workers
print("[%] Connecting to workers...")
ALICE = NodeClient(hook, "ws://localhost:3000")
BOB = NodeClient(hook, "ws://localhost:3001")
print("[+] Connected to workers")

print("[%] Create the serialized model object...")
dummy_input = th.empty(1, 1, 28, 28)
pytorch_model = ExampleNet(
model = OnnxModel.fromModel(pytorch_model, dummy_input).tag("crypten_model")
print("[+] Serialized model created")
    
print("[%] Sending the serialized model...")
alice_model_ptr = model.send(alice)
bob_model_ptr = model.send(bob)
print("[+] Model sent to alice and bob")
    
print("[%] Create some dummy data and send it to alice...")
data = th.tensor(dummy_input).tag("crypten_data")
data_ptr_alice = data.send(alice)
print("[+] Data sent to alice")

### Define the CrypTen computation

We need to specify for the ```run_multiworkers``` decorater:
* the workers that will take part in the computation
* the master address, this will be used for communication

We will use the ```func2plan``` decorator to:
* trace the operations from our function
* sending the plan operations to *alice* and *bob* - the plans operations will act as the function
* run the plans operations on both workers

In [ ]:
@run_multiworkers(
    [ALICE, BOB], master_addr="127.0.0.1"
)
@sy.func2plan()
def run_encrypted_inference(crypten=crypten):
    data = crypten.load("crypten_data", 0)

    # This should load the crypten model that is found at all parties
    model = crypten.load_model("crypten_model")

    model.encrypt()
    out = model(data)
    model.decrypt()
    out = out.get_plain_text()
    return out

## Run the CrypTen computation

Now let's run the inference

In [ ]:
# Get the returned values
# key 0 - return values for alice
# key 1 - return values for bob
print("[%] Starting computation")
func_ts = time()
result = run_encryptend_inference()[0]
func_te = time()
print(f"[+] run_encrypted_inference() took {int(func_te - func_ts)}s")

print(f"The label for data is {result}")

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on GitHub

The easiest way to help our community is just by starring the GitHub repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22Good+first+issue+%3Amortar_board%3A%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)